# Multiple Regression

In [1]:
import pandas as pd
base = pd.read_csv('../../../datasets/games.csv', encoding='ISO-8859-1')
base.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


## Preprocessing

In [2]:
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

Using TensorFlow backend.


In [3]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)

In [4]:
base = base.dropna(axis=0)

In [6]:
base = base.loc[base['NA_Sales'] > 1]
base = base.loc[base['EU_Sales'] > 1]

In [7]:
base['Name'].value_counts()

Grand Theft Auto V                              4
Call of Duty: Ghosts                            3
Assassin's Creed IV: Black Flag                 3
The Elder Scrolls V: Skyrim                     3
Battlefield 4                                   3
Call of Duty: Black Ops II                      2
Fallout: New Vegas                              2
Battlefield: Bad Company 2                      2
Call of Duty: Black Ops                         2
Battlefield 3                                   2
Call of Duty: Advanced Warfare                  2
Fallout 4                                       2
Assassin's Creed II                             2
Call of Duty 4: Modern Warfare                  2
Medal of Honor                                  2
Call of Duty: Modern Warfare 3                  2
Grand Theft Auto: Liberty City Stories          2
Call of Duty: Modern Warfare 2                  2
FIFA Soccer 13                                  2
Assassin's Creed                                2


In [8]:
name_games = base.Name

In [9]:
base = base.drop('Name', axis=1)

In [10]:
data = base.iloc[:, [0,1,2,3,7,8,9,10,11]].values
sales_na = base.iloc[:, 4].values
sales_eu = base.iloc[:, 5].values
sales_jp = base.iloc[:, 6].values

In [11]:
print(type(data))

<class 'numpy.ndarray'>


In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
data[:, 0] = labelencoder.fit_transform(data[:, 0])
data[:, 2] = labelencoder.fit_transform(data[:, 2])
data[:, 3] = labelencoder.fit_transform(data[:, 3])
data[:, 8] = labelencoder.fit_transform(data[:, 8])


In [17]:
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder(categorical_features = [0,2,3,8], n_values='auto')
data = onehotencoder.fit_transform(data ).toarray()


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [19]:
data

array([[   1. ,    0. ,    1. , ...,   51. ,    8. ,  322. ],
       [   1. ,    0. ,    1. , ...,   73. ,    8.3,  709. ],
       [   1. ,    0. ,    1. , ...,   73. ,    8. ,  192. ],
       ..., 
       [   1. ,    0. ,    1. , ...,   47. ,    8. ,  124. ],
       [   1. ,    0. ,    1. , ...,   15. ,    7.9,   46. ],
       [   1. ,    0. ,    1. , ...,   26. ,    8.4,   42. ]])

In [22]:
camada_entrada = Input(shape=(65,))
camada_oculta1 = Dense(units=32, activation='sigmoid')(camada_entrada)
camda_oculta2 = Dense(units=32, activation='sigmoid')(camada_oculta1)
camada_saida1 = Dense(units=1, activation='linear')(camda_oculta2)
camada_saida2 = Dense(units=1, activation='linear')(camda_oculta2)
camada_saida3 = Dense(units=1, activation='linear')(camda_oculta2)

regressor = Model(inputs = camada_entrada, outputs = [camada_saida1, camada_saida2, camada_saida3])

regressor.compile(optimizer = 'adam', loss = 'mse')
regressor.fit(data, [sales_na, sales_eu, sales_jp], epochs = 5000, batch_size = 100)
pred_na, pred_eu, pred_jp = regressor.predict(data)

Epoch 1/5000
258/258 [==============================] - 1s 3ms/step - loss: 33.2531 - dense_8_loss: 19.5902 - dense_9_loss: 12.4775 - dense_10_loss: 1.1855
Epoch 2/5000
258/258 [==============================] - 0s 43us/step - loss: 31.9139 - dense_8_loss: 18.8094 - dense_9_loss: 11.9302 - dense_10_loss: 1.1743
Epoch 3/5000
258/258 [==============================] - 0s 40us/step - loss: 30.6519 - dense_8_loss: 18.0603 - dense_9_loss: 11.4176 - dense_10_loss: 1.1740
Epoch 4/5000
258/258 [==============================] - 0s 60us/step - loss: 29.5245 - dense_8_loss: 17.4089 - dense_9_loss: 10.9418 - dense_10_loss: 1.1738
Epoch 5/5000
258/258 [==============================] - 0s 93us/step - loss: 28.3714 - dense_8_loss: 16.7284 - dense_9_loss: 10.4742 - dense_10_loss: 1.1688
Epoch 6/5000
258/258 [==============================] - 0s 52us/step - loss: 27.3269 - dense_8_loss: 16.1376 - dense_9_loss: 10.0212 - dense_10_loss: 1.1682
Epoch 7/5000
258/258 [==============================] - 0s 

258/258 [==============================] - 0s 44us/step - loss: 18.8202 - dense_8_loss: 11.8487 - dense_9_loss: 5.8396 - dense_10_loss: 1.1319
Epoch 54/5000
258/258 [==============================] - 0s 51us/step - loss: 18.8133 - dense_8_loss: 11.8437 - dense_9_loss: 5.8381 - dense_10_loss: 1.1315
Epoch 55/5000
258/258 [==============================] - 0s 39us/step - loss: 18.8165 - dense_8_loss: 11.8452 - dense_9_loss: 5.8393 - dense_10_loss: 1.1320
Epoch 56/5000
258/258 [==============================] - 0s 27us/step - loss: 18.8158 - dense_8_loss: 11.8456 - dense_9_loss: 5.8392 - dense_10_loss: 1.1311
Epoch 57/5000
258/258 [==============================] - 0s 32us/step - loss: 18.8145 - dense_8_loss: 11.8446 - dense_9_loss: 5.8391 - dense_10_loss: 1.1308
Epoch 58/5000
258/258 [==============================] - 0s 54us/step - loss: 18.8144 - dense_8_loss: 11.8441 - dense_9_loss: 5.8397 - dense_10_loss: 1.1306
Epoch 59/5000
258/258 [==============================] - 0s 36us/step - 

258/258 [==============================] - 0s 41us/step - loss: 18.7393 - dense_8_loss: 11.8048 - dense_9_loss: 5.8188 - dense_10_loss: 1.1157
Epoch 106/5000
258/258 [==============================] - 0s 47us/step - loss: 18.7383 - dense_8_loss: 11.8047 - dense_9_loss: 5.8183 - dense_10_loss: 1.1152
Epoch 107/5000
258/258 [==============================] - 0s 52us/step - loss: 18.7351 - dense_8_loss: 11.8021 - dense_9_loss: 5.8177 - dense_10_loss: 1.1152
Epoch 108/5000
258/258 [==============================] - 0s 46us/step - loss: 18.7360 - dense_8_loss: 11.8031 - dense_9_loss: 5.8180 - dense_10_loss: 1.1148
Epoch 109/5000
258/258 [==============================] - 0s 102us/step - loss: 18.7331 - dense_8_loss: 11.8011 - dense_9_loss: 5.8171 - dense_10_loss: 1.1149
Epoch 110/5000
258/258 [==============================] - 0s 48us/step - loss: 18.7333 - dense_8_loss: 11.8013 - dense_9_loss: 5.8171 - dense_10_loss: 1.1150
Epoch 111/5000
258/258 [==============================] - 0s 72us/

258/258 [==============================] - 0s 42us/step - loss: 18.6557 - dense_8_loss: 11.7657 - dense_9_loss: 5.7947 - dense_10_loss: 1.0952
Epoch 157/5000
258/258 [==============================] - 0s 76us/step - loss: 18.6584 - dense_8_loss: 11.7673 - dense_9_loss: 5.7938 - dense_10_loss: 1.0973
Epoch 158/5000
258/258 [==============================] - 0s 57us/step - loss: 18.6549 - dense_8_loss: 11.7643 - dense_9_loss: 5.7933 - dense_10_loss: 1.0973
Epoch 159/5000
258/258 [==============================] - 0s 51us/step - loss: 18.6482 - dense_8_loss: 11.7614 - dense_9_loss: 5.7927 - dense_10_loss: 1.0941
Epoch 160/5000
258/258 [==============================] - 0s 44us/step - loss: 18.6477 - dense_8_loss: 11.7624 - dense_9_loss: 5.7918 - dense_10_loss: 1.0935
Epoch 161/5000
258/258 [==============================] - 0s 68us/step - loss: 18.6456 - dense_8_loss: 11.7616 - dense_9_loss: 5.7912 - dense_10_loss: 1.0927
Epoch 162/5000
258/258 [==============================] - 0s 61us/s

258/258 [==============================] - 0s 42us/step - loss: 18.5835 - dense_8_loss: 11.7307 - dense_9_loss: 5.7704 - dense_10_loss: 1.0824
Epoch 208/5000
258/258 [==============================] - 0s 45us/step - loss: 18.5813 - dense_8_loss: 11.7293 - dense_9_loss: 5.7700 - dense_10_loss: 1.0820
Epoch 209/5000
258/258 [==============================] - 0s 54us/step - loss: 18.5779 - dense_8_loss: 11.7244 - dense_9_loss: 5.7699 - dense_10_loss: 1.0835
Epoch 210/5000
258/258 [==============================] - 0s 53us/step - loss: 18.5708 - dense_8_loss: 11.7224 - dense_9_loss: 5.7667 - dense_10_loss: 1.0817
Epoch 211/5000
258/258 [==============================] - 0s 44us/step - loss: 18.5657 - dense_8_loss: 11.7198 - dense_9_loss: 5.7642 - dense_10_loss: 1.0817
Epoch 212/5000
258/258 [==============================] - 0s 59us/step - loss: 18.5533 - dense_8_loss: 11.7115 - dense_9_loss: 5.7616 - dense_10_loss: 1.0803
Epoch 213/5000
258/258 [==============================] - 0s 40us/s

...

258/258 [==============================] - 0s 70us/step - loss: 10.0624 - dense_8_loss: 6.0186 - dense_9_loss: 3.2307 - dense_10_loss: 0.8131
Epoch 4746/5000
258/258 [==============================] - 0s 58us/step - loss: 10.0751 - dense_8_loss: 6.0114 - dense_9_loss: 3.2414 - dense_10_loss: 0.8223
Epoch 4747/5000
258/258 [==============================] - 0s 73us/step - loss: 10.1981 - dense_8_loss: 6.1064 - dense_9_loss: 3.2863 - dense_10_loss: 0.8053
Epoch 4748/5000
258/258 [==============================] - 0s 43us/step - loss: 10.0920 - dense_8_loss: 6.0257 - dense_9_loss: 3.2380 - dense_10_loss: 0.8282
Epoch 4749/5000
258/258 [==============================] - 0s 86us/step - loss: 10.3879 - dense_8_loss: 6.2570 - dense_9_loss: 3.3107 - dense_10_loss: 0.8202
Epoch 4750/5000
258/258 [==============================] - 0s 79us/step - loss: 10.2157 - dense_8_loss: 6.0884 - dense_9_loss: 3.2886 - dense_10_loss: 0.8387
Epoch 4751/5000
258/258 [==============================] - 0s 72us/s

Epoch 4797/5000
258/258 [==============================] - 0s 95us/step - loss: 9.9408 - dense_8_loss: 5.9164 - dense_9_loss: 3.2081 - dense_10_loss: 0.8163
Epoch 4798/5000
258/258 [==============================] - 0s 90us/step - loss: 9.9686 - dense_8_loss: 5.9377 - dense_9_loss: 3.2120 - dense_10_loss: 0.8189
Epoch 4799/5000
258/258 [==============================] - 0s 103us/step - loss: 9.9698 - dense_8_loss: 5.9367 - dense_9_loss: 3.2143 - dense_10_loss: 0.8188
Epoch 4800/5000
258/258 [==============================] - 0s 62us/step - loss: 9.9397 - dense_8_loss: 5.9300 - dense_9_loss: 3.1973 - dense_10_loss: 0.8124
Epoch 4801/5000
258/258 [==============================] - 0s 75us/step - loss: 9.9806 - dense_8_loss: 5.9360 - dense_9_loss: 3.2207 - dense_10_loss: 0.8239
Epoch 4802/5000
258/258 [==============================] - 0s 96us/step - loss: 9.9431 - dense_8_loss: 5.9145 - dense_9_loss: 3.2104 - dense_10_loss: 0.8183
Epoch 4803/5000
258/258 [==============================] 

258/258 [==============================] - 0s 53us/step - loss: 10.0024 - dense_8_loss: 5.9652 - dense_9_loss: 3.2271 - dense_10_loss: 0.8100
Epoch 4849/5000
258/258 [==============================] - 0s 37us/step - loss: 10.0792 - dense_8_loss: 5.9971 - dense_9_loss: 3.2663 - dense_10_loss: 0.8157
Epoch 4850/5000
258/258 [==============================] - 0s 59us/step - loss: 10.4275 - dense_8_loss: 6.1857 - dense_9_loss: 3.3695 - dense_10_loss: 0.8723
Epoch 4851/5000
258/258 [==============================] - 0s 87us/step - loss: 10.3946 - dense_8_loss: 6.2569 - dense_9_loss: 3.3277 - dense_10_loss: 0.8100
Epoch 4852/5000
258/258 [==============================] - 0s 59us/step - loss: 10.5848 - dense_8_loss: 6.3388 - dense_9_loss: 3.3655 - dense_10_loss: 0.8805
Epoch 4853/5000
258/258 [==============================] - 0s 50us/step - loss: 10.3404 - dense_8_loss: 6.1407 - dense_9_loss: 3.3792 - dense_10_loss: 0.8205
Epoch 4854/5000
258/258 [==============================] - 0s 86us/s

258/258 [==============================] - 0s 65us/step - loss: 10.0982 - dense_8_loss: 6.0716 - dense_9_loss: 3.2153 - dense_10_loss: 0.8113
Epoch 4901/5000
258/258 [==============================] - 0s 117us/step - loss: 10.3991 - dense_8_loss: 6.2749 - dense_9_loss: 3.3035 - dense_10_loss: 0.8207
Epoch 4902/5000
258/258 [==============================] - 0s 91us/step - loss: 10.1355 - dense_8_loss: 6.0655 - dense_9_loss: 3.2360 - dense_10_loss: 0.8340
Epoch 4903/5000
258/258 [==============================] - 0s 49us/step - loss: 10.0598 - dense_8_loss: 6.0076 - dense_9_loss: 3.2374 - dense_10_loss: 0.8148
Epoch 4904/5000
258/258 [==============================] - 0s 62us/step - loss: 10.1385 - dense_8_loss: 6.0438 - dense_9_loss: 3.2459 - dense_10_loss: 0.8489
Epoch 4905/5000
258/258 [==============================] - 0s 87us/step - loss: 10.6117 - dense_8_loss: 6.3349 - dense_9_loss: 3.4271 - dense_10_loss: 0.8497
Epoch 4906/5000
258/258 [==============================] - 0s 45us/

258/258 [==============================] - 0s 60us/step - loss: 9.9868 - dense_8_loss: 5.9573 - dense_9_loss: 3.2097 - dense_10_loss: 0.8198
Epoch 4953/5000
258/258 [==============================] - 0s 42us/step - loss: 9.9032 - dense_8_loss: 5.8881 - dense_9_loss: 3.1973 - dense_10_loss: 0.8178
Epoch 4954/5000
258/258 [==============================] - 0s 55us/step - loss: 10.0068 - dense_8_loss: 5.9968 - dense_9_loss: 3.1941 - dense_10_loss: 0.8159
Epoch 4955/5000
258/258 [==============================] - 0s 42us/step - loss: 9.9548 - dense_8_loss: 5.9447 - dense_9_loss: 3.1945 - dense_10_loss: 0.8156
Epoch 4956/5000
258/258 [==============================] - 0s 89us/step - loss: 9.9728 - dense_8_loss: 5.9402 - dense_9_loss: 3.2101 - dense_10_loss: 0.8225
Epoch 4957/5000
258/258 [==============================] - 0s 63us/step - loss: 9.9909 - dense_8_loss: 5.9442 - dense_9_loss: 3.2102 - dense_10_loss: 0.8365
Epoch 4958/5000
258/258 [==============================] - 0s 71us/step -

In [23]:
pred_na, pred_eu, pred_jp

(array([[ 15.82845688],
        [ 16.5887413 ],
        [ 14.80978203],
        [ 11.99992943],
        [  9.187253  ],
        [ 15.35818863],
        [ 11.07504845],
        [  9.46159554],
        [ 11.36032104],
        [  2.59956646],
        [  4.82105398],
        [  2.37990189],
        [  4.82105398],
        [  6.1040864 ],
        [  2.38232875],
        [  6.69901514],
        [  2.90671396],
        [  3.42439795],
        [  5.8611784 ],
        [  3.76468134],
        [  3.76468134],
        [  3.44318676],
        [  5.84776402],
        [  3.42439795],
        [  5.11687851],
        [  4.49170971],
        [  3.76468134],
        [  3.11585665],
        [  4.82105398],
        [  3.76468134],
        [  3.76468134],
        [  2.93047285],
        [  4.63004684],
        [  4.47127485],
        [  3.7646811 ],
        [  3.76468134],
        [  3.76467967],
        [  2.38156891],
        [  3.87467456],
        [  3.76468086],
        [  3.42439795],
        [  5.965